## ROI and Synapse Examples

This notebook demonstrates neuclease functions for:
- downloading synapse points from DVID as a CSV,
- downloading ROI geometry as a single combined label volume,
- and examining the volume to assign an ROI label to each synapse point.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import logging

import requests
from tqdm import tqdm_notebook
tqdm = tqdm_notebook

import numpy as np
import pandas as pd

In [3]:
from neuclease import configure_default_logging
configure_default_logging()

In [4]:
pwd

'/groups/flyem/proj/cluster/miniforge/envs/flyem/neuclease/notebooks'

### Choose UUID

In [5]:
from neuclease.dvid import find_master

server = 'emdata3:8900'
uuid = find_master(server)
uuid = uuid[:4]
print(f"Using master uuid: {uuid}")

master = (server, uuid)

Using master uuid: 7f0c


### Download synapses from DVID

In [6]:
from neuclease.dvid import fetch_synapses_in_batches

# See docs for details:
#help(fetch_synapses_in_batches)

# FULL DOWNLOAD
# For the full hemibrain, this will take an hour or so.
#synapses_df = fetch_synapses_in_batches(*master, 'synapses', 'segmentation', format='pandas')
#synapses_df.to_csv(f'synapses-{uuid}.csv', header=True, index=False)

# For testing, here's the same as above, but for only a small subvolume
box_zyx = [(16000,16000,16000), (17024,17024,17024)]
synapses_df = fetch_synapses_in_batches(*master, 'synapses', box_zyx, format='pandas')
synapses_df.to_csv(f'synapses-{uuid}-SUBVOLUME.csv', header=True, index=False)
synapses_df.head()

HBox(children=(IntProgress(value=0, max=256), HTML(value='')))

,x,y,z,kind,conf,user
0,16062,16050,16041,PostSyn,0.785155,$fpl
1,16120,16059,16041,PostSyn,0.955115,$fpl
2,16096,16059,16027,PostSyn,0.915969,$fpl
3,16100,16045,16051,PostSyn,0.956573,$fpl
4,16110,16055,16051,PostSyn,0.946751,$fpl


### Fetch/select ROI names

In [7]:
from neuclease.dvid import fetch_repo_instances

# Get the list of all ROIs
rois = fetch_repo_instances(*master, 'roi').keys()

# Select ROIs of interest (no '-lm' or 'L' rois)
rois = filter(lambda roi: not roi.startswith('(L)'), rois)
rois = filter(lambda roi: not roi.endswith('-lm'), rois)
rois = list(rois)
print(rois)

['AB', 'AL', 'AME', 'AMMC', 'AOT', 'AOTU', 'ATL', 'AVLP', 'BU', 'CA', 'CAN', 'CRE', 'EB', 'EPA', 'FB', 'FLA', 'GA', 'GC', 'GF', 'GNG', 'GOR', 'IB', 'ICL', 'IPS', 'LAL', 'LH', 'LO', 'LOP', 'ME', 'NO1', 'NO2D', 'NO2V', 'NO3', 'PB(L1)', 'PB(L2)', 'PB(L3)', 'PB(L4)', 'PB(L5)', 'PB(L6)', 'PB(L7)', 'PB(L8)', 'PB(L9)', 'PB(R1)', 'PB(R2)', 'PB(R3)', 'PB(R4)', 'PB(R5)', 'PB(R6)', 'PB(R7)', 'PB(R8)', 'PB(R9)', 'PED', 'PLP', 'POC', 'PRW', 'PVLP', 'ROB', 'RUB', 'SAD', 'SCL', 'SIP', 'SLP', 'SMP', 'SPS', 'VES', 'WED', "a'L", 'aL', "b'L", 'bL', 'dACA', 'gL', 'mALT', 'vACA']


### Load ROIs as a single volume

In [8]:
from neuclease.dvid import fetch_combined_roi_volume

# Fetch and combine into one volume
#roi_vol, box, overlaps = fetch_combined_roi_volume(*master, rois, box_zyx=[(0,0,0), None])

#### Alternatively, provide rois as a dict to explicitly choose the output label for each ROI

In [14]:
from neuclease.dvid import fetch_combined_roi_volume

rois = { roi_name: label for label, roi_name in enumerate(rois, start=1) }

# Combine all PB(*) rois into a single label
pb_label = rois['PB(L1)']
for name in rois.keys():
    if name.startswith('PB'):
        rois[name] = pb_label

roi_vol, box, overlaps = fetch_combined_roi_volume(*master, rois, box_zyx=[(0,0,0), None])
print(f"ROI volume consumes {roi_vol.nbytes / 1e9:.1f} GB of RAM")

HBox(children=(IntProgress(value=0, max=74), HTML(value='')))

HBox(children=(IntProgress(value=0, max=74), HTML(value='')))

ROI volume consumes 1.6 GB of RAM


### Assign ROI label to each point in a table

In [10]:
from neuclease.dvid import load_synapses # Knows proper dtypes for each CSV column

# Example: load a pre-downloaded synapses table
synapses_df = load_synapses('/nrs/flyem/bergs/complete-ffn-agglo/synapses-d585.csv')
print(f"Loaded {len(synapses_df)} points")

Loaded 71369749 points


In [11]:
from neuclease.util import extract_labels_from_volume

# Above, we forced all PB ROIs to a single label,
# so make sure the label names reflect that.
consolidated_rois = dict(rois)
consolidated_rois['PB'] = rois['PB(L1)']
for k in rois.keys():
    if k.startswith('PB'):
        del consolidated_rois[k]

# Scale is important here -- ROIs were downloaded at scale 5
extract_labels_from_volume(synapses_df, roi_vol, vol_scale=5, label_names=consolidated_rois)

INFO [2018-09-27 00:39:42,251] Extracting 71366455 ROI index values


In [12]:
synapses_df.head()

,x,y,z,kind,conf,user,label,label_name
0,23893,20166,1775,PostSyn,0.294397,$fpl,0,<unspecified>
1,23917,20161,1786,PreSyn,0.985000,$fpl,0,<unspecified>
2,23889,20453,1749,PostSyn,0.999742,$fpl,0,<unspecified>
3,23909,20469,1734,PostSyn,0.717127,$fpl,0,<unspecified>
4,23902,20458,1749,PostSyn,0.989156,$fpl,0,<unspecified>


In [13]:
synapses_df['label_name'].value_counts()

<unspecified>    11986353
LO                6223254
AVLP              4886147
SLP               4545272
SMP               3986675
PVLP              3965910
PLP               3372312
FB                2911311
AL                2841205
SPS               2147006
LH                1898021
WED               1867072
IB                1780529
ICL               1768989
SCL               1762854
LAL               1760440
SIP               1664827
VES               1412391
CRE               1123234
gL                1013228
EB                 887026
GNG                806847
IPS                697409
AOTU               692475
SAD                506368
CA                 488225
bL                 464947
aL                 439701
LOP                433161
AMMC               415769
b'L                374613
PB                 367222
ME                 297854
EPA                261628
PED                257061
a'L                224488
GOR                213373
ATL                174437
BU          